![](img/28_observability.png)

##### **<u>Согласно статье</u> 038 (Shauying R.K.) Observability of Nonlinear Systems**

<span style="color:#2b817d">Примечание:</span>     $km \geq n$

<span style="color:#0ab49a">Согласно статье</span> <span style="color:#A254FC">055 (Yujiro Inowe) On the Observability of Autonomous Nonlinear Systems</span> <span style="color:#0ab49a">надо просто проверить ранг во всём $R^n$!</span>    $rg\frac{\partial \boldsymbol{H}_d}{\partial \boldsymbol{x}} (\boldsymbol{x}) = n \hskip20px \forall x \in R^n,$
$$d = n \frac{n +3}{2}.$$

<span style="color:#0ab49a">Согласно статье</span> <span style="color:#A254FC">051 (Andrew J. Whalen) Observability and Controllability of Nonlinear Networks The Role of Symmetry</span> <span style="color:#0ab49a">надо проверить</span> $\delta(x) = \frac{|\sigma_{min}[O^T O]|}{|\sigma_{max}[O^T O]|}.$

###### <u>Инициализация</u>

In [1]:
from common_func import *
o, num_params, t, ω, μ, ρ, r_orb, v_orb = init_symbol_params()

Инициализация проекта kiam-formation
Высота орбиты: 500 км
Период орбиты: 1.57 часов
Плотность атмосферы: 5.264107768251325e-13 кг/м³


###### <u>Алгоритм</u> (новое)

In [2]:
def my_diff(expr, power: int = 1, n_c=1, n_d=1):
    """Функция дифференцирует выражение, подставля
    :param expr: Выражение, от которого надо взять производную по времени t
    :param power: Степень производной
    :param n_c: Количество материнских КА
    :param n_d: Количество дочерних КА
    """    
    global r_d, v_d, q_d, ω_d, r_c, v_c, q_c, ω_c
    global dr_d, dv_d, dq_d, dω_d, dr_c, dv_c, dq_c, dω_c
    if power == 0:
        return expr
    subses, anw = [], (expr if power == 1 else my_diff(expr, power - 1))
    for j in range(3):
        for i in range(n_d):
            subses.extend([(Derivative(r_d[i][j], t), dr_d[i][j]), (Derivative(v_d[i][j], t), dv_d[i][j]), 
                           (Derivative(q_d[i][j+1], t), dq_d[i][j+1]), (Derivative(ω_d[i][j], t), dω_d[i][j])])
        for i in range(n_c):
            subses.extend([(Derivative(r_c[i][j], t), dr_c[i][j]), (Derivative(v_c[i][j], t), dv_c[i][j]),
                           (Derivative(q_c[i][j+1], t), dq_c[i][j+1]), (Derivative(ω_c[i][j], t), dω_c[i][j])])
    anw = anw.diff(t).subs(subses)
    anw.simplify()
    return anw

def SubRandParams(o, J, n_d: int, n_c: int, n_x: int, n_y: int):
    """Берёт матрицу J размером n_x на n_y, подставляет случайные значения"""
    global r_d, v_d, q_d, ω_d, r_c, v_c, q_c, ω_c
    # Генерация случайных параметров движения
    s_r = lambda: o.v.spread(param='r', name="FemtoSat")[0]
    s_v = lambda: o.v.spread(param='v', name="FemtoSat")[0]
    s_w = lambda: o.v.spread(param='w', name="FemtoSat")[0]
    rand_params = [(ω, num_params['ω0']), (pi, np.pi), (ρ, num_params['ρ']), (v_orb, num_params['v0']), 
                   (o.f.c_resist, num_params['Cd']), (o.c.c_resist, num_params['Cc']), 
                   (o.f.mass, num_params['md']), (o.c.mass, num_params['mc']),
                   (o.f.size[0], num_params['sd'][0]), (o.f.size[1], num_params['sd'][1]),
                   (o.c.size[0], num_params['sc'][0]), (o.c.size[1], num_params['sc'][1]),
                   (o.f.J[0, 0], num_params['Jd'][0, 0]), (o.f.J[1, 1], num_params['Jd'][1, 1]), (o.f.J[2, 2], num_params['Jd'][2, 2])] 
    for i in range(n_d):
        q = np.random.uniform(-1, 1, 4)
        for j in range(3):
            _r, _v, _w, _q = [o.v.specific_initial[f"ChipSat {c}"][j] for c in ['r','v','w','q-vec']] if i==0 else [s_r(), s_v(), s_w(), q[j+1] / np.linalg.norm(q)]
            rand_params.extend([(r_d[i][j], _r), (v_d[i][j], _v), (??_d[i][j], _w), (q_d[i][j+1], _q)])
    for i in range(n_c):
        q = np.random.uniform(-1, 1, 4)
        for j in range(3):
            _r, _v, _w, _q = [o.v.specific_initial[f"CubeSat {c}"][j] for c in ['r','v','w','q-vec']] if i==0 else [s_r(), s_v(), s_w(), q[j+1] / np.linalg.norm(q)]
            rand_params.extend([(r_c[i][j], _r), (v_c[i][j], _v), (??_c[i][j], _w), (q_c[i][j+1], _q)])

    # Якобиан матрицы наблюдаемости: J[измерение (H), состояние (X)] 
    J_numb = np.array([[float(J[i, j].subs(rand_params).subs(t,0)) for j in range(n_x)] for i in range(n_y)])
    return J_numb

def ShauyingObservabilitySufficientCondition(o, n_d: int, n_c: int, X: list, Y: list, hand_written_deriv: int = None, is_only_xz: bool = False):
    """Проверка достаточного условия наблюдаемости системы. Проверка равномерного отношения миноров матрицы наблюдаемости.
    :param n_d: Количество чипсатов
    :param X: Список неизвестных параметров, которые необходимо найти
    :param Y: Список известных параметров (измерений системы в t₀=0)
    :param my_diff: Функция взятия производной по времени"""
    global r_d, v_d, q_d, ω_d, r_c, v_c, q_c, ω_c
    def print_and_record(report: str, lcl_txt: str):
        print(lcl_txt)
        return report + lcl_txt + "\n"
    report = kf.my_print(f"Количество кубсатов: {n_c}\nКоличество чипсатов: {n_d}\n", if_return=True, bold=True)

    # Количество одномоментных измерений
    l = len(Y)
    # Требуемое количество существующих производных функции измерения
    k = int(len(X) // len(Y)) if hand_written_deriv is None else hand_written_deriv
    txt = f"" if hand_written_deriv is None else f"\033[1mВнимание! Рассчитывается не отношение миноров, а ранг расширенного Якобиана\033[0m\n"
    report = print_and_record(report, txt + f"Неизвестные: n = {len(X)} (на каждый чипсат по {int(len(X) // n_d)} параметров)\nИзвестные: l = {l}\n∃ производные порядка k = {len(X) / len(Y)} (Должна быть целой!)")

    # >>>>>>>> РАССЧЁТ ОТОБРАЖЕНИЯ НАБЛЮДАЕМОСТИ H <<<<<<<< #
    if is_only_xz:
        f = Matrix([Matrix([*dr_d[i][::2], *dv_d[i][::2]]) for i in range(len(dr_d))])
    else:
        f = Matrix([Matrix([*dr_d[i], *dv_d[i]]) for i in range(len(dr_d))])
        # f = Matrix([Matrix([*dr_d[i], *dq_d[i][1:4], *dv_d[i], *dω_d[i]]) for i in range(len(dr_d))]) if o.v.NAVIGATION_ANGLES else Matrix([Matrix([*dr_d[i], *dv_d[i]]) for i in range(len(dr_d))])
    h, H = Y, []
    for kk in range(k):
        if kk > 0:
            dhdx = Matrix([[h[ll].diff(X[j]) for j in range(len(X))] for ll in range(l)])
            dhdx_f = dhdx @ f
            dhdx_f.simplify()
            for ll in range(l):
                print(f"__расчёт матрицы H_: k={(kk+1)}/{k}, l={(ll+1)}/{l}")
                h[ll] = my_diff(h[ll], n_c=n_c, n_d=n_d) + dhdx_f[ll]
        H.extend(h)
    H = Matrix(H)
    report = print_and_record(report, f"Размерность матрицы H: {shape(H)}")

    # Якобиан матрицы наблюдаемости: J[измерение (H), состояние (X)]
    J = Matrix([[H[i].diff(X[j]) for j in range(len(X))] for i in range(k * l)])
    report = print_and_record(report, f"Размерность матрицы J: {shape(J)}")

    # Подстановка конкретных значений
    J_numb = SubRandParams(o=o, J=J, n_c=n_c, n_d=n_d, n_x=len(X), n_y=k*l)
    _, v, _ = np.linalg.svd(J_numb.T @ J_numb)
    report = print_and_record(report, f"v = {v}")
    report = print_and_record(report, f"σₘₙ/σₘₐₓ = {np.min(v)}/{np.max(v)} = {np.min(v) / np.max(v)} | σ>10⁻⁷: {np.sum(v>1e-7)}/{len(v)} (f-051)")

    # Достаточное условие
    txt = f"\nРанг матрицы: {[np.linalg.matrix_rank(J_numb, tol=tol) for tol in [1e-3, 1e-5, 1e-7, 1e-10, 1e-12, 1e-15]]} (f-055)\n"
    txt += f"Детерминант матрицы: {np.linalg.det(J_numb)}\n" if J_numb.shape[0] == J_numb.shape[1] else ""
    if hand_written_deriv is None:
        A = np.eye(J_numb.shape[0])
    else:
        A = np.repeat(np.eye(len(X)), k, axis=0).reshape(len(X), len(X)*k)[:,:len(Y)*k]
    report = print_and_record(report, txt + f"Следующие параметры не должны быть нулевыми:\n")
    d, Δ, flag, i_min = [], [], True, -1
    print(f"A: {A.shape}, J: {J_numb.shape}")
    for i in range(len(X)):
        tmp = kf.principal_minor(A @ J_numb, i)
        d += [tmp if i == 0 else tmp / Δ[-1]]
        Δ += [tmp]
        report = print_and_record(report, f"Δ_{i} = {d[-1]}" if i == 0 else f"Δ_{i} / Δ_{i-1} = {d[-1]}")
    
        # Чек наблюдаемости
        if flag and abs(d[-1]) < 1e-10:
            i_min = i
            flag = False
        if not flag:
            break

    # Вывод
    if flag:
        txt = f"\n\033[1mВыполнено достаточное условие! Система наблюдаема\033[0m"
    else:
        in_txt = f"Δ_{i_min}" if i_min == 0 else f"Δ_{i_min} / Δ_{i_min-1}"
        txt = f"\n\033[1mНе выполнено достаточное условие. Нулевой параметр: {in_txt} = {d[i_min]}\033[0m"
    # report = print_and_record(report, txt)
    # return H, J, J_numb, Δ, report
    report = print_and_record(report, txt)
    return H, J, J_numb, None, report
    

def observe_system(n_c, n_d, is_d_qw, gains, is_drag, is_only_xz, hand_written_deriv, disp: bool = True, Hcheck: bool = True):
    global r_d, v_d, q_d, ω_d, r_c, v_c, q_c, ω_c, t
    global dr_d, dv_d, dq_d, dω_d, dr_c, dv_c, dq_c, dω_c, dydq
    
    o.v.GAIN_MODEL_C_N = gains['cN']
    o.v.GAIN_MODEL_F_N = gains['dN']
    o.v.init_choice_params()
    o.c.gain_mode = o.v.GAIN_MODEL_C = o.v.GAIN_MODES[gains['cN']]
    o.f.gain_mode = o.v.GAIN_MODEL_F = o.v.GAIN_MODES[gains['dN']]
    o.v.NAVIGATION_ANGLES = is_d_qw
    o.v.DYNAMIC_MODEL['aero drag'] = is_drag
    o.v.MULTI_ANTENNA_TAKE = True
    o.v.MULTI_ANTENNA_SEND = True
    o.v.ANCHOR.r_irf[0] = Matrix([r_orb, 0, 0])
    o.v.P = r_orb  # * (1 - self.ECCENTRICITY**2)  # Фокальный параметр
    o.v.MU = μ
    o.v.W_ORB = ω
    o.v.R_ORB = r_orb
    o.v.V_ORB = v_orb
    o.v.RHO = ρ
    
    r_d, v_d, q_d, ω_d = get_state_vector(func=kf.get_func, obj='d', n=n_d)
    r_c, v_c, q_c, ω_c = get_state_vector(func=kf.get_func, obj='c', n=n_c)

    o.c.n = n_c
    o.f.n = n_d
    o.f.r_orf, o.f.v_orf, o.f.q, o.f.w_brf = r_d, v_d, q_d, ω_d
    o.c.r_orf, o.c.v_orf, o.c.q, o.c.w_brf = r_c, v_c, q_c, ω_c

    x = []
    for i in range(n_d):
        x.extend(r_d[i])
        if is_d_qw:
            x.extend(q_d[i][1:4])
        x.extend(v_d[i])
        if is_d_qw:
            x.extend(ω_d[i])
    x = Matrix(x)
    
    kf.my_print(f"Вектор состояния:", bold=True)
    display(x.T)

    if is_only_xz:
        tmp = []
        for i in range(n_d):
            tmp.extend([r_d[i][0], r_d[i][2], v_d[i][0], v_d[i][2]])
        x = Matrix(tmp)
        kf.my_print(f"Вектор состояния (изменённый):", bold=True)
        display(x.T)

    dr_d, dv_d, dq_d, dω_d = ([0 for _ in range(n_d)] for _ in range(4))
    dr_c, dv_c, dq_c, dω_c = ([0 for _ in range(n_c)] for _ in range(4))
    for i in range(n_d):
        dr_d[i], dv_d[i] = kf.translate_rhs(vrs=o.v, obj=o.f, i=i, rv=(r_d[i], v_d[i]), w=ω, mu=μ, rho=ρ)
        dq_d[i], dω_d[i] = kf.attitude_rhs(vrs=o.v, obj=o.f, t=t, i=i, qw=(q_d[i], ω_d[i]))
    for i in range(n_c):
        dr_c[i], dv_c[i] = kf.translate_rhs(vrs=o.v, obj=o.c, i=i, rv=(r_c[i], v_c[i]), w=ω, mu=μ, rho=ρ)
        dq_c[i], dω_c[i] = kf.attitude_rhs(vrs=o.v, obj=o.c, t=t, i=i, qw=(q_c[i], ω_c[i]))
    if disp:
        kf.my_print(f"Динамика: \n{' '*10}dr = ", bold=True)
        display(dr_d[0])
        kf.my_print(f"{' '*10}dv = ", bold=True)
        display(dv_d[0])
        kf.my_print(f"{' '*10}dq = ", bold=True)
        display(dq_d[0])
        # kf.my_print(f"{' '*10}dω = ", bold=True)
    
    y, _, notes = kf.measure_antennas_power(c=o.c, f=o.f, vrs=o.v, p=o.p, j=int(len(x) // n_d), estimated_params=x, t=t)

    # >>>>>>>>>>>>>>>>> Проверка матрицы H <<<<<<<<<<<<<<<<<
    if Hcheck:
        H_1 = kf.h_matrix(t=t, v=o.v, f=o.f, c=o.c, r_f=r_d, r_c=r_c, 
                          q_f=[Matrix(q_d[i][1:4]) for i in range(o.f.n)], q_c=[Matrix(q_c[i][1:4]) for i in range(o.c.n)])
        H_2 = zeros(*H_1.shape)
        for ix, xx in enumerate(x):
            for iy, yy in enumerate(y):
                H_2[iy, ix] = yy.diff(xx)
        kf.my_print("Проверка матрицы H:", bold=True)
        tmp = H_1 - H_2
        tmp.simplify()
        display(tmp)
    
    return ShauyingObservabilitySufficientCondition(o=o, n_c=n_c, n_d=n_d, X=x, Y=y, hand_written_deriv=hand_written_deriv, is_only_xz=is_only_xz)

###### <u>Case №1</u>: No?

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=True, 
                                     gains={'cN': 0, 'dN': 0},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, n_d=2, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=True, 
                                     gains={'cN': 0, 'dN': 0},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

###### <u>Case №2</u>: Yes?

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=2, n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=True, 
                                     gains={'cN': 0, 'dN': 0},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

###### <u>Case №3</u>: Yes?

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=3, n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=False, 
                                     gains={'cN': 0, 'dN': 0},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

###### <u>Case №4</u>: Yes?

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=True, 
                                     is_only_xz=True, 
                                     gains={'cN': 0, 'dN': 0},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

###### <u>Case №5</u>: Yes?

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=2, n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=True, 
                                     is_only_xz=False, 
                                     gains={'cN': 0, 'dN': 0},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

###### <u>Case №6</u>: Yes?

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=False, 
                                     gains={'cN': 3, 'dN': 0},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

###### <u>Case №7</u>: Yes?

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=True, 
                                     is_only_xz=False, 
                                     gains={'cN': 3, 'dN': 0},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

###### <u>Case №8</u>: No?

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=True, 
                                     is_only_xz=False, 
                                     gains={'cN': 3, 'dN': 0},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

###### <u>Case №9</u>: Yes?

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=True, 
                                     is_only_xz=False, 
                                     gains={'cN': 3, 'dN': 0},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=2, 
                                     n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=True, 
                                     is_only_xz=False, 
                                     gains={'cN': 0, 'cModel': 'isotropic', 'dN': 0, 'dModel': 'isotropic'},
                                     hand_written_deriv=None, disp=True, Hcheck=True)

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, 
                                     n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=True, 
                                     is_only_xz=True, 
                                     gains={'cN': 0, 'cModel': 'isotropic', 'dN': 0, 'dModel': 'isotropic'},
                                     hand_written_deriv=None, disp=True, Hcheck=True)

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=2, 
                                     n_d=5, 
                                     is_d_qw=False, 
                                     is_drag=True, 
                                     is_only_xz=True, 
                                     gains={'cN': 0, 'cModel': 'isotropic', 'dN': 0, 'dModel': 'isotropic'},
                                     hand_written_deriv=None, disp=True, Hcheck=True)

##### **<u>Влияние закрутки КА на наблюдаемость</u>**

###### <u>Инициализация</u>

In [1]:
from common_func import *

n_c, n_d = 1, 1
angles = True

o, num_params, t, w_0, μ, ρ, r_orb, v_orb = init_symbol_params()
o.f.n = n_d
o.c.n = n_c
o.f.r_orf, o.f.v_orf, o.f.q, o.f.w_irf = get_state_vector(func=kf.get_vars, obj='d', n=n_d)
o.c.r_orf, o.c.v_orf, o.c.q, o.c.w_irf = get_state_vector(func=kf.get_vars, obj='c', n=n_c)
o.c.gain_mode = o.v.GAIN_MODEL_C = '2 antennas'
o.f.gain_mode = o.v.GAIN_MODEL_F = 'isotropic'
o.v.N_ANTENNA_C = 2
o.v.N_ANTENNA_F = 1
o.v.NAVIGATION_ANGLES = True
o.v.MULTI_ANTENNA_SEND = True
o.v.MULTI_ANTENNA_TAKE = True

Инициализация проекта kiam-formation
Параметры не могут быть загружены! Нет файла: kiamformation/data/config_choose.csv
Матрицы Ф:(6, 6), Q:(3, 3), P:(6, 6), D:(6, 3)
Высота орбиты: 400 км
Период орбиты: 1.54 часов
Плотность атмосферы: 6.404751331738951e-12 кг/м³


In [2]:
kf.measure_antennas_power(c=o.c, f=o.f, v=o.v, noise=0, produce=True, p=o.p, estimated_params=[])
o.v.MEASURES_VECTOR_NOTES
# o.v.MEASURES_VECTOR

['cf 0 0 0 0 2 1', 'cf 0 0 1 0 2 1']

###### <u>Исследование частных случаев</u> (без учёта динамики)

In [3]:
params_to_sub = []

# Движения в плоскости орбиты
params_to_sub += [(o.f.r_orf[0][1], 0), (o.c.r_orf[0][1], 0),
                  (o.f.v_orf[0][1], 0), (o.c.v_orf[0][1], 0),]

# Ориентация в плоскости орбиты
# params_to_sub += [(o.f.q[0][1], 0), (o.c.q[0][1], 0),
#                   (o.f.q[0][2], 0), (o.c.q[0][2], 0),]

# Кубсат неподвижен в ОСК
params_to_sub += [(o.c.r_orf[0][0], 0), (o.c.r_orf[0][1], 0), (o.c.r_orf[0][2], 0), 
                  (o.c.q[0][1], 0), (o.c.q[0][2], 0), (o.c.q[0][3], 0), ]

y = o.v.MEASURES_VECTOR.subs(params_to_sub)
y

Matrix([
[1.0*sqrt(r_0^d_x**2 + r_0^d_z**2)*(1.0*r_0^d_x**2 + 1.0*r_0^d_z**2)**(3/4)/Abs(r_0^d_x)**(3/2)],
[1.0*sqrt(r_0^d_x**2 + r_0^d_z**2)*(1.0*r_0^d_x**2 + 1.0*r_0^d_z**2)**(3/4)/Abs(r_0^d_z)**(3/2)]])

In [21]:
y1 = y[0].simplify()
y2 = y[1].simplify()

kf.my_print(f"Измерения:", bold=True)
display(y[0])
display(y[1])

kf.my_print(f"Чё то 1:", bold=True)
tmp = (1/y1**4 + 1/y2**4).simplify()
display(tmp)

kf.my_print(f"Чё то 2:", bold=True)
tmp = ((y1 / y2)**4).simplify()
display(tmp)

kf.my_print(f"Чё то 3:", bold=True)
t1 = 1/y1
t2 = 1/y2
tmp = (t1 / (t1**2 + t2**2)).simplify()
display(tmp)

Измерения:


1.0*sqrt(r_0^d_x**2 + r_0^d_z**2)*(1.0*r_0^d_x**2 + 1.0*r_0^d_z**2)**(3/4)/Abs(r_0^d_x)**(3/2)

1.0*sqrt(r_0^d_x**2 + r_0^d_z**2)*(1.0*r_0^d_x**2 + 1.0*r_0^d_z**2)**(3/4)/Abs(r_0^d_z)**(3/2)

Чё то 1:


0.999999999999999*(Abs(r_0^d_x**(3/2))**4 + Abs(r_0^d_z**(3/2))**4)/(r_0^d_x**2 + r_0^d_z**2)**5

Чё то 2:


1.0*r_0^d_z**6/r_0^d_x**6

Чё то 3:


1.0*(r_0^d_x**2 + r_0^d_z**2)**(5/4)*Abs(r_0^d_x**(3/2))/(Abs(r_0^d_x**(3/2))**2 + Abs(r_0^d_z**(3/2))**2)

*Та не получается ниче без динамики*

###### <u>Исследование частных случаев</u> (с учётом динамики)

In [3]:
params_to_sub = []

# Движения в плоскости орбиты
params_to_sub += [(o.f.r_orf[0][1], 0), (o.c.r_orf[0][1], 0),
                  (o.f.v_orf[0][1], 0), (o.c.v_orf[0][1], 0),]

# Ориентация в плоскости орбиты
# params_to_sub += [(o.f.q[0][1], 0), (o.c.q[0][1], 0),
#                   (o.f.q[0][2], 0), (o.c.q[0][2], 0),]

# Кубсат неподвижен в ОСК
params_to_sub += [(o.c.r_orf[0][0], 0), (o.c.r_orf[0][1], 0), (o.c.r_orf[0][2], 0), 
                  (o.c.q[0][1], 0), (o.c.q[0][2], 0), (o.c.q[0][3], 0), ]

# Динамика
c_f = kf.get_c_hkw(o.f.r_orf[0], o.f.v_orf[0], w_0)
r_f = kf.r_hkw(c_f, w_0, t)
params_to_sub += [(o.f.r_orf[0][0], r_f[0]), (o.f.r_orf[0][1], r_f[1]), (o.f.r_orf[0][2], r_f[2])]

y = o.v.MEASURES_VECTOR.subs(params_to_sub)
y

Matrix([
[1.0*sqrt((4*r_0^d_z + 2*v_0^d_x/ω_0 + v_0^d_z*sin(t*ω_0)/ω_0 + (-3*r_0^d_z - 2*v_0^d_x/ω_0)*cos(t*ω_0))**2 + (r_0^d_x + t*ω_0*(-24*r_0^d_z - 15*v_0^d_x/ω_0 - 6*v_0^d_z*sin(t*ω_0)/ω_0 - 6*(-3*r_0^d_z - 2*v_0^d_x/ω_0)*cos(t*ω_0)) + 2*v_0^d_z*cos(t*ω_0)/ω_0 - 2*v_0^d_z/ω_0 - (-24*r_0^d_z - 16*v_0^d_x/ω_0 - 6*v_0^d_z*sin(t*ω_0)/ω_0 - 6*(-3*r_0^d_z - 2*v_0^d_x/ω_0)*cos(t*ω_0))*sin(t*ω_0))**2)*(1.0*(4*r_0^d_z + 2*v_0^d_x/ω_0 + v_0^d_z*sin(t*ω_0)/ω_0 + (-3*r_0^d_z - 2*v_0^d_x/ω_0)*cos(t*ω_0))**2 + 1.0*(r_0^d_x + t*ω_0*(-24*r_0^d_z - 15*v_0^d_x/ω_0 - 6*v_0^d_z*sin(t*ω_0)/ω_0 - 6*(-3*r_0^d_z - 2*v_0^d_x/ω_0)*cos(t*ω_0)) + 2*v_0^d_z*cos(t*ω_0)/ω_0 - 2*v_0^d_z/ω_0 - (-24*r_0^d_z - 16*v_0^d_x/ω_0 - 6*v_0^d_z*sin(t*ω_0)/ω_0 - 6*(-3*r_0^d_z - 2*v_0^d_x/ω_0)*cos(t*ω_0))*sin(t*ω_0))**2)**(3/4)/Abs(r_0^d_x + t*ω_0*(-24*r_0^d_z - 15*v_0^d_x/ω_0 - 6*v_0^d_z*sin(t*ω_0)/ω_0 - 6*(-3*r_0^d_z - 2*v_0^d_x/ω_0)*cos(t*ω_0)) + 2*v_0^d_z*cos(t*ω_0)/ω_0 - 2*v_0^d_z/ω_0 - (-24*r_0^d_z - 16*v_0^d_x/ω_0 - 6*

**Симметрия относительно уравнений движения**

In [50]:
t1_n, t1_d = fraction(t1**2)  # Разделение на числитель и знаменатель
t2_n, t2_d = fraction(t2**2)

kf.my_print(f"Равные знаменатели:", bold=True)
display((t1_d - t2_d).simplify())

kf.my_print(f"Сумма числителей:", bold=True)
tmp_01 = ((t1_n + t2_n) ).simplify()
display(tmp_01)

Равные знаменатели:


0

Сумма числителей:


1.0*Abs((4*r_0^d_z*ω_0 + 2*v_0^d_x + v_0^d_z*sin(t*ω_0) - (3*r_0^d_z*ω_0 + 2*v_0^d_x)*cos(t*ω_0))/ω_0)**3 + 1.0*Abs((2*v_0^d_z*cos(t*ω_0) - 2*v_0^d_z + ω_0*(r_0^d_x - 3*t*(8*r_0^d_z*ω_0 + 5*v_0^d_x + 2*v_0^d_z*sin(t*ω_0) - (6*r_0^d_z*ω_0 + 4*v_0^d_x)*cos(t*ω_0))) + 2*(12*r_0^d_z*ω_0 + 8*v_0^d_x + 3*v_0^d_z*sin(t*ω_0) - (9*r_0^d_z*ω_0 + 6*v_0^d_x)*cos(t*ω_0))*sin(t*ω_0))/ω_0)**3